In [ ]:
pip install asyncio

In [ ]:
import os
import nest_asyncio
import asyncio
import json
from pathlib import Path
from dotenv import load_dotenv
from proxy_lite import Runner, RunnerConfig
from transformers import AutoProcessor
from qwen_vl_utils import process_vision_info
from proxy_lite.tools import ReturnValueTool, BrowserTool
from proxy_lite.serializer import OpenAICompatibleSerializer

# Apply nest_asyncio to support async calls
nest_asyncio.apply()

# Load environment variables
load_dotenv()

# Ensure running in the correct directory
if "src" not in os.listdir():
    os.chdir("..")

# Load tasks from JSON file
TASKS_FILE = "benchmarktasks.json"  # Change this if your JSON file is named differently
with open(TASKS_FILE, "r", encoding="utf-8") as f:
    tasks = json.load(f)

# Load processor and tools
processor = AutoProcessor.from_pretrained("convergence-ai/proxy-lite-3b")
tools = OpenAICompatibleSerializer().serialize_tools([ReturnValueTool(), BrowserTool(session=None)])

async def run_agent(task):
    """Runs the agent for a given task, dynamically setting the URL and objective."""

    # Dynamically configure Proxy Lite Runner for each task
    config = RunnerConfig.from_dict(
        {
            "environment": {
                "name": "webbrowser",
                "homepage": task["url"],  # Set dynamically based on the task
                "headless": False,
            },
            "solver": {
                "name": "simple",
                "agent": {
                    "name": "proxy_lite",
                    "client": {
                        "name": "convergence",
                        "model_id": "convergence-ai/proxy-lite-3b",
                        "api_base": "https://convergence-ai-demo-api.hf.space/v1",
                    },
                },
            },
            "max_steps": 50,
            "action_timeout": 1800,
            "environment_timeout": 1800,
            "task_timeout": 18000,
            "logger_level": "DEBUG",
        },
    )

    # Initialize Proxy Lite Runner with dynamic config
    proxy = Runner(config=config)

    # Construct message history dynamically
    message_history = [
        {
            "role": "system",
            "content": "You are Proxy Lite, a web-browsing agent that can perform searches, extract information, and take actions based on observations. Use the browser tool to interact with the web.",
        },
        {
            "role": "user",
            "content": task["objective"],  # Replace with task objective
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": f"URL: {task['url']}"}  # Replace with task URL
            ],
        },
    ]

    # Process message history
    templated_messages = processor.apply_chat_template(
        message_history, tokenize=False, add_generation_prompt=True, tools=tools
    )

    image_inputs, video_inputs = process_vision_info(message_history)

    # Prepare batch for model input
    batch = processor(
        text=[templated_messages],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )

    # Run the agent
    response = await proxy.run(templated_messages)

    print(f"Task {task['step']} completed.\nResponse:\n{response}\n")
    return response

async def main():
    """Runs the agent sequentially on all tasks."""
    for task in tasks:
        await run_agent(task)

# Run the agent for all tasks
asyncio.run(main())